In [2]:
import pandas as pd
import json
import minsearch as mn
from openai import OpenAI
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch


In [4]:
op_client =  OpenAI()
es_client = Elasticsearch('http://localhost:9200')

In [5]:
with open('/workspaces/LLM_Zoomcamp/file_documents/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)


documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [100]:
mn_index = mn.Index(text_fields=['question','text','section'],
                 keyword_fields=['course'])

mn_index.fit(documents)

In [7]:
mn_index = mn.Index(text_fields=['question','text','section'],
                 keyword_fields=['course'])

mn_index.fit(documents)mn_index = mn.Index(text_fields=['question','text','section'],
                 keyword_fields=['course'])

mn_index.fit(documents)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questiions'})

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [101]:
def min_search(query):
    boost = {'question':3.0, 'section':0.5}

    result = mn_index.search(
    query = query,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
    )

    return result

In [102]:
def build_prompt(query, seach_result):
    prompt_template = """
    You're a course assitant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the  answer, output NONE

    QUESTION : {question}

    CONTEXT : {context}
    """.strip()
    context = ""

    for doc in seach_result:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"


    prompt = prompt_template.fornat(queston=query, context=context).strip()

    return prompt

def llm(prompt):
    response = op_client.chat.completions.create(
    model = 'gpt-4o',
    message = [{"role":"user", "content":prompt}]
    )
    return response.choices[0].message.content




In [ ]:
def es_searrch(query):
    es_search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, size=es_search_query['size'], query=es_search_query['query'])

    result_doc = []

    for doc in response['hits']['hits']:
        result_doc.append(doc['_source'])

    
    return result_doc

In [118]:
def rag(query):
    search_result = min_search(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)
    return search_result

def rag2(query):
    search_result = es_searrch(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)
    return search_result

In [122]:
query = 'do i need docker knowlge for this course'

In [123]:
mnresponse = rag(query)
print(mnresponse[0])

{'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repository on your computer will make it easy for you to access the instructors’ code and make pull requests (if you want to add your own notes or make changes to the course content).\nYou will probably also create your own repositories that host your notes, versions of your file, to do this. Here is a great tutorial that shows you how to do this: https://www.atlassian.com/git/tutorials/setting-up-a-repository\nRemember to ignore large database, .csv, and .gz files, and other files that should not be saved to a repository. Use .gitignore for this: https://www.atlassian.com/git/tutorials/saving-changes/gitignore NEVER store passwords or keys in a git repo (even if that repo is set to private).\nThis is also a great resource: https://dangitgit.com/', 'section': 'General cou

In [124]:
esresponse = rag2(query)
print(esresponse[0])

{'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repository on your computer will make it easy for you to access the instructors’ code and make pull requests (if you want to add your own notes or make changes to the course content).\nYou will probably also create your own repositories that host your notes, versions of your file, to do this. Here is a great tutorial that shows you how to do this: https://www.atlassian.com/git/tutorials/setting-up-a-repository\nRemember to ignore large database, .csv, and .gz files, and other files that should not be saved to a repository. Use .gitignore for this: https://www.atlassian.com/git/tutorials/saving-changes/gitignore NEVER store passwords or keys in a git repo (even if that repo is set to private).\nThis is also a great resource: https://dangitgit.com/', 'section': 'General cou